### ***Wczytanie danych***
https://www.kaggle.com/datasets/mylesoneill/world-university-rankings

Dane które wybraliśmy dotyczą rankingu uczelni wyższych. Celem analizy jest wyłonić najlepszy uniwersytet w ramach całego świata.

In [ ]:
import numpy as np
import pandas as pd


df = pd.read_csv("cwurData.csv", index_col=False)

df.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


### ***Opis zmiennych***

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   world_rank            2200 non-null   int64  
 1   institution           2200 non-null   object 
 2   country               2200 non-null   object 
 3   national_rank         2200 non-null   int64  
 4   quality_of_education  2200 non-null   int64  
 5   alumni_employment     2200 non-null   int64  
 6   quality_of_faculty    2200 non-null   int64  
 7   publications          2200 non-null   int64  
 8   influence             2200 non-null   int64  
 9   citations             2200 non-null   int64  
 10  broad_impact          2000 non-null   float64
 11  patents               2200 non-null   int64  
 12  score                 2200 non-null   float64
 13  year                  2200 non-null   int64  
dtypes: float64(2), int64(10), object(2)
memory usage: 240.8+ KB


###*Podział zmiennych na kategoryczne / numeryczne*

In [ ]:
categorical = [var for var in df.columns if df[var].dtype == 'O']
numerical = [var for var in df.columns if df[var].dtype != 'O']

df[categorical].isnull().sum()

institution    0
country        0
dtype: int64

In [ ]:
df[numerical].isnull().sum()

world_rank                0
national_rank             0
quality_of_education      0
alumni_employment         0
quality_of_faculty        0
publications              0
influence                 0
citations                 0
broad_impact            200
patents                   0
score                     0
year                      0
dtype: int64

###*Usunięcie rekordów które zawierają niepoprawną wartość w kolumnie 'broad_impact'*

In [ ]:
df = df.dropna(axis=0, subset=['broad_impact'])
df[numerical].isnull().sum()

world_rank              0
national_rank           0
quality_of_education    0
alumni_employment       0
quality_of_faculty      0
publications            0
influence               0
citations               0
broad_impact            0
patents                 0
score                   0
year                    0
dtype: int64

###*Teraz nasz data-set jest już oczyszczony ze zbędnych rekordów*

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 200 to 2199
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   world_rank            2000 non-null   int64  
 1   institution           2000 non-null   object 
 2   country               2000 non-null   object 
 3   national_rank         2000 non-null   int64  
 4   quality_of_education  2000 non-null   int64  
 5   alumni_employment     2000 non-null   int64  
 6   quality_of_faculty    2000 non-null   int64  
 7   publications          2000 non-null   int64  
 8   influence             2000 non-null   int64  
 9   citations             2000 non-null   int64  
 10  broad_impact          2000 non-null   float64
 11  patents               2000 non-null   int64  
 12  score                 2000 non-null   float64
 13  year                  2000 non-null   int64  
dtypes: float64(2), int64(10), object(2)
memory usage: 234.4+ KB


###*Oddzielenie kolumny która determinuję pozycję uczelni od reszty zmiennych*

In [ ]:
x = df.drop(['world_rank'], axis=1)
y = df['world_rank']

###*Podział na zbiory treningowy / testowy + Trenowanie modelu*

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Podzial na zbior treningowy i testowy (np. 80% danych do treningu, 20% do testow)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicjalizacja modelu regresji losowego lasu (Random Forest Regressor)
model = RandomForestRegressor(random_state=42)

# Trenowanie modelu na danych treningowych
model.fit(X_train, y_train)

# Przewidywanie na danych testowych
predictions = model.predict(X_test)

# Obliczanie bledu sredniokwadratowego (MSE) jako miary jakosci modelu
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)
